## Import libraries

In [2]:
import numpy as np
import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

## Read data

In [5]:
data_uschad = pd.read_table("../data/USC-HAD/UCS-HAD_cleared.txt", delimiter=',', header=None)
data_uschad.columns = ['id_user', 'activity', 'timestamp', 'x', 'y', 'z']

data_wisdm = pd.read_table("../data/WISDM/WISDM_ar_v1.1_raw_cleared.txt", delimiter=',', header=None)
data_wisdm.columns = ['id_user', 'activity', 'timestamp', 'x', 'y', 'z']

## Creating object-feature matrix

So we need to construct 10 seconds time series. To do it we need to remember the following:
* each time series should be from one user and one type of activity;
* in the time series timestamp should't differ more then 0.2 second (empirical rule, in ideal all timestamp should differ on 50 ms = 0.05 second).

Now let's create **object-feature** matrix:

In [9]:
def check_candidate(candidate, data_type, threshold=2.*1e8):
    if data_type == "USCHAD":
        threshold = 0.
    tsp = np.array(candidate['timestamp'])
    diffs = tsp[1:] - tsp[:-1]
    
    return np.sum(diffs > threshold) == 0

def get_time_series(accelerations, data_type, nb=200):
    accelerations.index = [i for i in range(len(accelerations))]
    TS = []
    st = 0
    fi = st + nb
    while fi < len(accelerations):
        candidate = accelerations.loc[[st + i for i in range(nb)], :]
        if check_candidate(candidate, data_type):
            TS.append([np.array(candidate['x']), 
                       np.array(candidate['y']), 
                       np.array(candidate['z'])])
        st = fi
        fi += nb
    
    return TS

In [21]:
def get_distribution(data, df):
    classes = list(set(data['activity']))
    for activity in classes:
        nb = np.sum(df['activity'] == classes.index(activity))
        print("{:<20}{:<9d}{:<5.2f} %".format(activity, nb, 100. * nb / df.shape[0]))
    print("")
    print("Number of objects: {:d}".format(df.shape[0]))

In [59]:
def get_feature_matrix(data, data_type, get_feature_names, get_features, params=[]):
    
    classes = list(set(data['activity']))
    feature_names = get_feature_names(params)
    df = pd.DataFrame(columns=['activity']+feature_names) 

    id_range = np.unique(np.array(data['id_user']))
    for id_user in id_range:
        for activity in classes:
            mask = (data.loc[:, 'id_user'] == id_user) & (data.loc[:, 'activity'] == activity)
            accelerations = data.loc[mask, ['timestamp', 'x', 'y', 'z']].copy()
            TS = get_time_series(accelerations, data_type, nb=200)
            for ts in TS:
                features = get_features(ts, params)
                df.loc[len(df), :] = [classes.index(activity)] + features
    return df

## Feature extraction

### Expert functions

The idea is the following: we will consider 10 seconds time series (or 200 points of measurements) and calculate 40 features:
* ```[3]``` - mean acceleration of each axis;
* ```[3]``` - std of acceleration of each axis;
* ```[3]``` - mean absolute deviation of acceleration of each axis;
* ```[1]``` - mean acceleration;
* ```[30]``` - distribution of time series values of each axis. First of all we calculate min and max of each component ($X, Y, Z$) from the whole interval. Then we divide the range of values of each component into 10 equal intervals and calculate on each each interval the percent of values that are in it (in the corresponding interval).  

And apply LogisticRegression and SVM.

In [60]:
def get_expert_names(params):
    feature_names = ['avg_x', 'avg_y', 'avg_z', 
                     'std_x', 'std_y', 'std_z', 
                     'abs_x', 'abs_y', 'abs_z', 'mean']
    for i in range(10):
        name = str(i) + '_'
        feature_names += [name + 'x', name + 'y', name + 'z']
        
    return feature_names

def get_expert_features(ts, params):
    x = ts[0]
    y = ts[1]
    z = ts[2]
    n = x.shape[0]
    features = []
    features.append(x.mean())
    features.append(y.mean())
    features.append(z.mean())
    features.append(x.std())
    features.append(y.std())
    features.append(z.std())
    features.append(np.abs(x - x.mean()).mean())
    features.append(np.abs(y - y.mean()).mean())
    features.append(np.abs(z - z.mean()).mean())
    features.append((x+y+z).mean() / 3.)
    x_range = np.linspace(x.min(), x.max(), 11)
    y_range = np.linspace(y.min(), y.max(), 11)
    z_range = np.linspace(z.min(), z.max(), 11)
    for i in range(10):
        features.append(1. * np.sum((x_range[i] <= x) & (x < x_range[i+1])) / n)
        features.append(1. * np.sum((y_range[i] <= y) & (y < y_range[i+1])) / n)
        features.append(1. * np.sum((z_range[i] <= z) & (z < z_range[i+1])) / n)
    
    return features

Example:

In [61]:
df_expert_wisdm = get_feature_matrix(data_wisdm, 'WISDM', get_expert_names, get_expert_features)
get_distribution(data_wisdm, df_expert_wisdm)

Standing            229      5.30  %
Walking             1917     44.36 %
Upstairs            466      10.78 %
Sitting             277      6.41  %
Jogging             1075     24.88 %
Downstairs          357      8.26  %

Number of objects: 4321


In [62]:
df_expert_uschad = get_feature_matrix(data_uschad, 'USCHAD', get_expert_names, get_expert_features)
get_distribution(data_uschad, df_expert_uschad)

Standing            1167     8.57  %
Elevator-up         764      5.61  %
Walking-forward     1874     13.76 %
Sitting             1294     9.50  %
Walking-downstairs  951      6.98  %
Sleeping            1860     13.66 %
Elevator-down       763      5.60  %
Walking-upstairs    1018     7.47  %
Jumping             495      3.63  %
Walking-right       1305     9.58  %
Walking-left        1280     9.40  %
Running             849      6.23  %

Number of objects: 13620


### Autoregression model

In [53]:
def get_autoregressive_names(params):
    n = params[0]
    feature_names = []
    for ax in ['x', 'y', 'z']:
        feature_names += ['intercept_' + ax]
        for i in range(n):
            feature_names += ['coef_' + str(i) + '_' + ax]
            
    return feature_names

def get_autoregressive_features(ts, params):
    n = params[0]
    x = ts[0]
    y = ts[1]
    z = ts[2]
    m = x.shape[0]
    features = []
    X = np.zeros([m-n, n])
    Y = np.zeros(m-n)
    for axis in [x, y, z]:
        for i in range(m-n):
            X[i, :] = axis[i:i+n]
            Y[i] = axis[i+n]
        lr = LinearRegression()
        lr.fit(X, Y)
        features.append(lr.intercept_)
        features.extend(lr.coef_)
    
    return features

Example:

In [63]:
df_ar_wisdm = get_feature_matrix(data_wisdm, 'WISDM', get_autoregressive_names,
                                 get_autoregressive_features, params)
get_distribution(data_wisdm, df_ar_wisdm)

Standing            229      5.30  %
Walking             1917     44.36 %
Upstairs            466      10.78 %
Sitting             277      6.41  %
Jogging             1075     24.88 %
Downstairs          357      8.26  %

Number of objects: 4321


In [64]:
df_ar_uschad = get_feature_matrix(data_uschad, 'USCHAD', get_autoregressive_names,
                                  get_autoregressive_features, params)
get_distribution(data_uschad, df_ar_uschad)

Standing            1167     8.57  %
Elevator-up         764      5.61  %
Walking-forward     1874     13.76 %
Sitting             1294     9.50  %
Walking-downstairs  951      6.98  %
Sleeping            1860     13.66 %
Elevator-down       763      5.60  %
Walking-upstairs    1018     7.47  %
Jumping             495      3.63  %
Walking-right       1305     9.58  %
Walking-left        1280     9.40  %
Running             849      6.23  %

Number of objects: 13620


### Spectrum analysis

In [66]:
def get_spectrum_names(params):
    n = params[0]
    feature_names = []
    for ax in ['x', 'y', 'z']:
        for i in range(n):
            feature_names += ['eigv_' + str(i) + '_' + ax]
            
    return feature_names

def get_spectrum_features(ts, params):
    n = params[0]
    x = ts[0]
    y = ts[1]
    z = ts[2]
    m = x.shape[0]
    features = []
    X = np.zeros([m-n, n])
    Y = np.zeros(m-n)
    for axis in [x, y, z]:
        for i in range(m-n):
            X[i, :] = axis[i:i+n]
        h = sc.linalg.svd(X.T.dot(X), compute_uv=False, overwrite_a=True)
        features.extend(h)
    
    return features

Example:

In [67]:
df_ssa_wisdm = get_feature_matrix(data_wisdm, 'WISDM', get_spectrum_names,
                                  get_spectrum_features, params)
get_distribution(data_wisdm, df_ssa_wisdm)

Standing            229      5.30  %
Walking             1917     44.36 %
Upstairs            466      10.78 %
Sitting             277      6.41  %
Jogging             1075     24.88 %
Downstairs          357      8.26  %

Number of objects: 4321


In [68]:
df_ssa_uschad = get_feature_matrix(data_uschad, 'USCHAD', get_spectrum_names,
                                   get_spectrum_features, params)
get_distribution(data_uschad, df_ssa_uschad)

Standing            1167     8.57  %
Elevator-up         764      5.61  %
Walking-forward     1874     13.76 %
Sitting             1294     9.50  %
Walking-downstairs  951      6.98  %
Sleeping            1860     13.66 %
Elevator-down       763      5.60  %
Walking-upstairs    1018     7.47  %
Jumping             495      3.63  %
Walking-right       1305     9.58  %
Walking-left        1280     9.40  %
Running             849      6.23  %

Number of objects: 13620


## Classification

In [70]:
def get_internal_score(clf, X, y, max_iter=20):
    nb = np.unique(y).shape[0]
    scores = np.zeros(nb+1)
    for j in range(max_iter):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        clf.fit(X_train, y_train)
        y_predict = clf.predict(X_test)
        scores[0] += accuracy_score(y_test, y_predict)
        for i in range(nb):
            scores[i+1] += accuracy_score(1*(np.array(y_test) == i), 
                                          1*(np.array(y_predict) == i))
            
    return scores / max_iter

In [71]:
def get_score(df, estimator, params_grid, test_size=0.3):
    X = df.iloc[:, 1:].values
    y = df['activity'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    clf = GridSearchCV(estimator, params_grid)
    clf.fit(X_train, list(y_train))
    clf_lr = clf.best_estimator_
    scores = get_internal_score(clf_lr, X, list(y))
    
    return scores

## Testing part 

In [73]:
parameters = {'penalty': ['l1', 'l2'], 
              'class_weight': ['balanced', None], 
              'C': 10. ** np.arange(0, 4, 1)}

scores_wisdm = {}
scores_uschad = {}

**Expert** features:

In [ ]:
df_expert_wisdm = get_feature_matrix(data_wisdm, 'WISDM', get_expert_names, get_expert_features)
df_expert_uschad = get_feature_matrix(data_uschad, 'USCHAD', get_expert_names, get_expert_features)

In [ ]:
scores_wisdm['lr_expert'] = get_score(df_expert_wisdm, LogisticRegression(), parameters)
scores_uschad['lr_expert'] = get_score(df_expert_uschad, LogisticRegression(), parameters)

From **autoregression model** features:

In [ ]:
params = [10]
n = params[0]

df_ar_wisdm = get_feature_matrix(data_wisdm, 'WISDM', get_autoregressive_names,
                                 get_autoregressive_features, params)
df_ar_uschad = get_feature_matrix(data_uschad, 'USCHAD', get_autoregressive_names, 
                                  get_autoregressive_features, params)

In [17]:
scores_wisdm['lr_ar_' + str(n)] = get_score(df_ar_wisdm, LogisticRegression(), parameters)
scores_uschad['lr_ar_' + str(n)] = get_score(df_ar_uschad, LogisticRegression(), parameters)

From **spectrum analysis** features:

In [ ]:
params = [10]
n = params[0]

df_ssa_wisdm = get_feature_matrix(data_wisdm, 'WISDM', get_spectrum_names,
                                 get_spectrum_features, params)
df_ssa_uschad = get_feature_matrix(data_uschad, 'USCHAD', get_spectrum_names, 
                                  get_spectrum_features, params)

In [17]:
scores_wisdm['lr_ssa_' + str(n)] = get_score(df_ssa_wisdm, LogisticRegression(), parameters)
scores_uschad['lr_ssa_' + str(n)] = get_score(df_ssa_uschad, LogisticRegression(), parameters)

## Results 

In [ ]:
results_wisdm = pd.DataFrame.from_dict(scores_wisdm, orient='index')
results_wisdm.columns = ['all'] + list(set(data_wisdm['activity']))

results_uschad = pd.DataFrame.from_dict(scores_uschad, orient='index')
results_uschad.columns = ['all'] + list(set(data_uschad['activity']))

In [ ]:
results_wisdm

In [ ]:
results_uschad